In [150]:
import pandas as pd
import numpy as np

In [151]:
Full_ICD10_ATC = pd.read_csv("/home/kabank/workbench/.conda/analysis/kabank-data/new-data/Full_ICD10_ATC.csv")
Full_ICD10_ATC.columns


Index(['episode_id', 'num_diagnoses', 'num_medications', 'pasient', 'age',
       'remaining_time_countdown', 'var_no_dates_permonth', 'gender',
       'episode_order', 'islast', 'closingcode', 'aftercode',
       'episode_start_date', 'episode_end_date', 'tillnextepisode',
       'Length_of_Episode', 'Cat_LOE', 'TNE_BO_180', 'TNE_NO_180',
       'TNE_BO_365', 'TNE_NO_365', 'TNE_BO_730', 'TNE_NO_730', 'TNE_BO_1095',
       'TNE_NO_1095', 'Cat_LOE_desc', 'Count_visit', 'Cat_CV', 'Therapy_ratio',
       'Examination_ratio', 'Advisory_ratio', 'TreatmentPlanning_ratio',
       'Outpatient_ratio', 'Inpatient_ratio', 'Inpatient_day_ratio',
       'Inpatient_daynight_ratio', 'Care_intensity', 'age_group',
       'closingcode_0', 'closingcode_1', 'closingcode_2', 'closingcode_3',
       'closingcode_4', 'closingcode_5', 'closingcode_6', 'closingcode_9',
       'aftercode_1', 'aftercode_2', 'aftercode_3', 'aftercode_4',
       'aftercode_5', 'gender_0', 'F', 'M', 'MiddleChildhood', 'Preschooler

In [152]:
len(Full_ICD10_ATC)

17125

In [153]:
diagnoses_ICD10_df = pd.read_csv( "/home/kabank/workbench/.conda/analysis/Resources/Diagnosis.CSV", decimal=',')
# Remove duplicates from diagnoses_ICD10_df
print(len(diagnoses_ICD10_df)) #42803 episodes

diagnoses_ICD10_list_df = diagnoses_ICD10_df.groupby('episode_id').agg({
    'diagnosis': lambda x: list(set(x))
})
diagnoses_ICD10_list_df = diagnoses_ICD10_list_df.reset_index()
diagnoses_ICD10_list_df.rename(columns={'diagnosis': 'ICD_codes'}, inplace=True)
#diagnoses_ICD10_list_df.info() # 17949 episodes
print(diagnoses_ICD10_list_df.head(3))

42803
  episode_id           ICD_codes
0       10.0  [F900, F431, F942]
1    10001.0         [X6n0, T4n]
2    10002.0              [F900]


In [154]:
# Identify non-integer values
non_integer_values = diagnoses_ICD10_list_df[~diagnoses_ICD10_list_df['episode_id'].astype(str).str.isdigit()]

# Convert non-integer values to NaN and then fill NaN with a specific integer value
diagnoses_ICD10_list_df['episode_id'] = pd.to_numeric(diagnoses_ICD10_list_df['episode_id'], errors='coerce')
diagnoses_ICD10_list_df['episode_id'] = diagnoses_ICD10_list_df['episode_id'].fillna(-1).astype(int)


In [155]:
Full_ICD10_ATC['episode_id'] = Full_ICD10_ATC['episode_id'].astype(float) 
diagnoses_ICD10_list_df['age'] = diagnoses_ICD10_list_df['episode_id'].astype(int) 

Full_ICD10_ATC = pd.merge(Full_ICD10_ATC, diagnoses_ICD10_list_df[['episode_id', 'ICD_codes']], on='episode_id', how='outer')

Full_ICD10_ATC.columns
#len(Full_ICD10_ATC)

Index(['episode_id', 'num_diagnoses', 'num_medications', 'pasient', 'age',
       'remaining_time_countdown', 'var_no_dates_permonth', 'gender',
       'episode_order', 'islast', 'closingcode', 'aftercode',
       'episode_start_date', 'episode_end_date', 'tillnextepisode',
       'Length_of_Episode', 'Cat_LOE', 'TNE_BO_180', 'TNE_NO_180',
       'TNE_BO_365', 'TNE_NO_365', 'TNE_BO_730', 'TNE_NO_730', 'TNE_BO_1095',
       'TNE_NO_1095', 'Cat_LOE_desc', 'Count_visit', 'Cat_CV', 'Therapy_ratio',
       'Examination_ratio', 'Advisory_ratio', 'TreatmentPlanning_ratio',
       'Outpatient_ratio', 'Inpatient_ratio', 'Inpatient_day_ratio',
       'Inpatient_daynight_ratio', 'Care_intensity', 'age_group',
       'closingcode_0', 'closingcode_1', 'closingcode_2', 'closingcode_3',
       'closingcode_4', 'closingcode_5', 'closingcode_6', 'closingcode_9',
       'aftercode_1', 'aftercode_2', 'aftercode_3', 'aftercode_4',
       'aftercode_5', 'gender_0', 'F', 'M', 'MiddleChildhood', 'Preschooler

In [156]:
Full_ICD10_ATC = Full_ICD10_ATC.dropna(subset=['age'])
Full_ICD10_ATC['age'] = Full_ICD10_ATC['age'].astype(int) 
Full_ICD10_ATC['episode_id'] = Full_ICD10_ATC['episode_id'].astype(int) 
Full_ICD10_ATC.rename(columns={'pasient': 'patient'}, inplace=True)
Full_ICD10_ATC['patient'] = Full_ICD10_ATC['patient'].astype(int) 
Full_ICD10_ATC['var_no_dates_permonth'] = Full_ICD10_ATC['var_no_dates_permonth'].astype(float).round(2)


In [157]:
Full_ICD10_ATC[['patient', 'episode_id','episode_start_date', 'episode_end_date', 'gender', 'age', 'var_no_dates_permonth', 'actual_med_Full_ATC', 'ICD_codes' ]]

,patient,episode_id,episode_start_date,episode_end_date,gender,age,var_no_dates_permonth,actual_med_Full_ATC,ICD_codes
0,2,3836,2013-07-02,2015-07-03,F,12,1.04,NaN,"[F930, F431]"
1,6,24660,2004-10-01,2007-07-06,M,6,1.23,NaN,NaN
2,7,22380,2015-09-28,2016-10-06,M,8,1.46,NaN,[H932]
3,8,28188,2002-12-30,2005-01-09,F,16,1.13,NaN,[F321]
4,10,7441,2008-12-29,2009-03-30,F,17,1.58,NaN,NaN
...,...,...,...,...,...,...,...,...,...
17120,19732,27636,2003-12-29,2012-10-05,M,9,0.70,NaN,"[F901, F952]"
17121,19744,30915,2002-12-30,2003-10-03,F,12,0.51,NaN,NaN
17122,19745,14014,2010-01-04,2013-04-04,F,14,0.91,"['N06BA09', 'R06AD01']","[F901, F928, H101]"
17123,19805,2894,1994-01-04,1994-07-04,M,13,0.65,NaN,NaN


In [158]:
Full_ICD10_ATC[['patient', 'episode_id','episode_start_date', 'episode_end_date', 'gender', 'age', 'var_no_dates_permonth', 'actual_med_Full_ATC', 'ICD_codes' ]].to_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/PatientExplorer1.csv', index=False)